In [1]:
import tensorflow as tf

In [2]:
root_path = "/tf/home/sergio/Tesis"

In [3]:
import sys
sys.path.append(root_path+"/TinyYOLOv3-Pedestrian-Detection")
from YOLOproduction import TinyYOLOv3,BasicBlock,PredictionLayer#,YOLOLossBasicBlock

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import tensorflow as tf

from skimage.io import imread,imshow
from skimage.transform import resize
import time
import os

from pycocotools.coco import COCO

In [4]:
anchors =[[0.02078,0.049],[0.0426,0.128],[0.08523,0.19356],[0.1506,0.4163],[0.27835,0.58651],[0.5632,0.78614]]
model = TinyYOLOv3(anchor_boxes=anchors)
model.build(batch_input_shape=(None,416,416,3))    
model.load_weights(root_path+'/weights_saved/pesos_finetuning_5521_30_50_epoch_nadam_0dot00001_msle_exp_mse_3anchors')

In [5]:
model.summary()

Model: "tiny_yol_ov3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
BasicBlock1 (BasicBlock)     multiple                  496       
_________________________________________________________________
BasicBlock2 (BasicBlock)     multiple                  4736      
_________________________________________________________________
BasicBlock3 (BasicBlock)     multiple                  18688     
_________________________________________________________________
BasicBlock4 (BasicBlock)     multiple                  74240     
_________________________________________________________________
BasicBlock5 (BasicBlock)     multiple                  295936    
_________________________________________________________________
BasicBlock6 (BasicBlock)     multiple                  1181696   
_________________________________________________________________
BasicBlock7 (BasicBlock)     multiple                 

In [6]:
file = tf.keras.utils.get_file(
    "persons.jpg",
    "https://www.saltwire.com/media/photologue/photos/cache/STJ-A01-28102019-RawlinsCrossPedestrians3_large.jpg")

In [7]:
image = file
img_raw = tf.image.decode_image(open(image, 'rb').read(), channels=3)
img = tf.expand_dims(img_raw, 0)
_,height,width,_ =img.shape
img = tf.image.resize(img, (416, 416))/255

In [8]:
boxes,objectness = model(img)

In [16]:
boxes = boxes.numpy()

In [17]:
%timeit output = boxes[0,np.where(objectness.numpy()>0.5),:]

13.8 µs ± 32.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [18]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model_production/my_model') 

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model_production/my_model/assets


In [19]:
new_model = tf.keras.models.load_model('saved_model_production/my_model')
# Check its architecture
new_model.summary()

Model: "tiny_yol_ov3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
BasicBlock1 (BasicBlock)     multiple                  496       
_________________________________________________________________
BasicBlock2 (BasicBlock)     multiple                  4736      
_________________________________________________________________
BasicBlock3 (BasicBlock)     multiple                  18688     
_________________________________________________________________
BasicBlock4 (BasicBlock)     multiple                  74240     
_________________________________________________________________
BasicBlock5 (BasicBlock)     multiple                  295936    
_________________________________________________________________
BasicBlock6 (BasicBlock)     multiple                  1181696   
_________________________________________________________________
BasicBlock7 (BasicBlock)     multiple                 

In [23]:
%timeit new_model(img)

3.67 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
1/0.00367

272.47956403269757

In [26]:
tiempo = []
with tf.device("CPU:0"):
    for i in range(2000):
        inicio = time.time()
        _ = new_model(img)
        fin = time.time()
        tiempo.append(fin-inicio)
print(1/np.mean(tiempo))

41.9716319150932
